# Search Engine With Tools and Agents

In [1]:
!pip install wikipedia


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=45f8071243b23d3804a14b94398c73339ee91b20f11010b88193a4ac232d86cb
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [2]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=21438c39a4c934272efe10b2c199d8ba19aa2c846ae74059a3b99e778eb38652
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [19]:
## Importing tools and API Wrapper 
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import  ArxivAPIWrapper,WikipediaAPIWrapper

In [20]:
# Used the inbuild tools of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# Making the wikipedia tool with name wiki
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki


WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\kumar\\anaconda3\\envs\\ann_project\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [21]:
# Used the inbuild tools of Arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# Making the Arxiv tool with name arxiv
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [22]:
# Make the list of tools
tools = [wiki,arxiv]

In [23]:
#Make the custom Tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
import os
from dotenv import load_dotenv
# Load all the environment variables
load_dotenv()

True

In [25]:
os.environ['HF_TOKEN'] = os.getenv("HUGGING_FACE_TOKEN")

In [29]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [31]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.9 MB 3.7 MB/s eta 0:00:04
   -------- ------------------------------- 3.1/14.9 MB 4.5 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.9 MB 3.7 MB/s eta 0:00:04
   ------------- -------------------------- 5.0/14.9 MB 4.2 MB/s eta 0:00:03
   --------------- ------------------------ 5.8/14.9 MB 4.2 MB/s eta 0:00:03
   ---------------- ----------------------- 6.0/14.9 MB 3.8 MB/s eta 0:00:03
   ------------------- -------------------- 7.1/14.9 MB 3.9 MB/s eta 0:00:03
   ------------------- -------------------- 7.3/14.9 MB 3.6 MB/s eta 0:00:03
   --------------------- ------------------ 8.1/14.9 MB 3.7 MB/s eta 0:00:02
   ----------------------- ---------------- 8.7/14.9 MB 3.7 MB/s eta 0:00:02
   ---

In [33]:

#Making the documents 
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
#Store in the vetor Database 
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"))
#Making the retriever from the vector database
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020B82237560>, search_kwargs={})

In [34]:
## Convert retriever as a tool

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [35]:
tools = [wiki, arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\kumar\\anaconda3\\envs\\ann_project\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020BD90D0F40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000